In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# To run cross validation parallelized
from joblib import Parallel, delayed

2024-06-17 17:49:27.603609: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 17:49:27.605998: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 17:49:27.652242: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 17:49:27.653351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-17 17:49:28.433334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate_parallelized(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
    n_jobs=-1,  # Set the number of parallel jobs, -1 for using all available cores
):
    def process_fold(train_index, test_index, dates, cement_types, x, y):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        set_seeds(SEED + REPEAT)

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        return score

    scores = Parallel(n_jobs=n_jobs)(
        delayed(process_fold)(train_index, test_index, dates, cement_types, x, y)
        for train_index, test_index in cv.split(x)
    )

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]

    results = [scores_final]
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class LSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
SEED = 47
REPEAT = 0

<IPython.core.display.Javascript object>

In [14]:
def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [15]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [16]:
def suppress_warnings():
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

<IPython.core.display.Javascript object>

In [17]:
def supress_other_stuff():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    # os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

<IPython.core.display.Javascript object>

In [18]:
suppress_warnings()
supress_other_stuff()

<IPython.core.display.Javascript object>

In [19]:
index_to_save = 2

<IPython.core.display.Javascript object>

In [20]:
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = ["Cement_Type_CP III-32"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [21]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "209",
    "Plant": "E",
    "Features": "Chemical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "LSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [22]:
df = pd.read_csv("../../../../../../data/processed/209/e.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [23]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "Blaine",        
        "Final setting time",
        "Initial setting time",
        "CS3",
        "CS7",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [24]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [25]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - LSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [26]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate_parallelized(
        LSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.000 (0.047)
MAE: 1.456 (0.046)
MAPE: 0.033 (0.001)
R2: 0.564 (0.020)


******
[TEST]
******
RMSE: 2.125 (0.190)
MAE: 1.541 (0.131)
MAPE: 0.035 (0.003)
R2: 0.500 (0.084)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.683 (0.243)
MAE: 1.340 (0.233)
MAPE: 0.030 (0.005)
R2: 0.685 (0.089)


******
[TEST]
******
RMSE: 1.991 (0.226)
MAE: 1.560 (0.195)
MAPE: 0.035 (0.004)
R2: 0.529 (0.091)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.393 (0.145)
MAE: 1.089 (0.131)
MAPE: 0.025 (0.003)
R2: 0.784 (0.047)


******
[TEST]
******
RMSE: 1.869 (0.181)
MAE: 1.466 (0.169)
MAPE: 0.033 (0.004)
R2: 0.490 (0.133)


Minutes Elapsed:  5.67171360651652


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model Cross Validation Timesteps RMSE Test            MAE Test  \
                                                  mean       std      mean   
0  Chemical  LSTM   Repeated KFold         1  2.124806  0.190481  1.540817   
1  Chemical  LSTM   Repeated KFold         7  1.991031  0.226499  1.560172   
2  Chemical  LSTM   Repeated KFold        14  1.869274  0.180537  1.465891   

            MAPE Test             R2 Test            
        std      mean       std      mean       std  
0  0.131379  0.035177  0.003031  0.499794  0.084028  
1  0.194801  0.035212  0.004446  0.528595  0.091456  
2  0.168536  0.033069  0.003750  0.489726  0.133338

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [28]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for repeat in range(repeats):
        REPEAT = repeat
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate_parallelized(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 4.777 (0.210)
MAE: 3.788 (0.217)
MAPE: 0.085 (0.004)
R2: -3.588 (3.023)


******
[TEST]
******
RMSE: 8.856 (2.877)
MAE: 6.513 (1.524)
MAPE: 0.146 (0.033)
R2: -65.474 (74.654)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.811 (0.319)
MAE: 1.499 (0.314)
MAPE: 0.034 (0.007)
R2: 0.324 (0.543)


******
[TEST]
******
RMSE: 8.607 (4.011)
MAE: 6.680 (2.959)
MAPE: 0.152 (0.069)
R2: -47.407 (54.517)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.791 (0.444)
MAE: 1.469 (0.395)
MAPE: 0.033 (0.009)
R2: 0.381 (0.523)


******
[TEST]
******
RMSE: 8.184 (3.143)
MAE: 7.167 (2.912)
MAPE: 0.161 (0.064)
R2: -44.044 (53.454)


Minutes Elapsed:  2.423005215326945


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation Timesteps RMSE Test            \
                                                             mean       std   
0  Chemical  LSTM  Blocking Time Series Split         1  8.855760  2.876757   
1  Chemical  LSTM  Blocking Time Series Split         7  8.606517  4.011269   
2  Chemical  LSTM  Blocking Time Series Split        14  8.183504  3.142711   
3  Chemical  LSTM              Repeated KFold         1  2.124806  0.190481   
4  Chemical  LSTM              Repeated KFold         7  1.991031  0.226499   
5  Chemical  LSTM              Repeated KFold        14  1.869274  0.180537   

   MAE Test           MAPE Test              R2 Test             
       mean       std      mean       std       mean        std  
0  6.512965  1.523500  0.146088  0.032531 -65.473778  74.653616  
1  6.680117  2.959418  0.151559  0.069241 -47.407059  54.516751  
2  7.167373  2.911560  0.160932  0.064122 -44.043854  53.454247  
3  1.540817  0.131379  0.035177  0.003031   0.499794   0.084028  
4  1.560172  0.194801  0.035212  0.004446   0.528595   0.091456  
5  1.465891  0.168536  0.033069  0.003750   0.489726   0.133338

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [30]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for repeat in range(repeats):
        REPEAT = repeat
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate_parallelized(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.727 (0.938)
MAE: 2.068 (0.838)
MAPE: 0.046 (0.017)
R2: -1.251 (3.428)


******
[TEST]
******
RMSE: 6.829 (6.591)
MAE: 4.221 (3.149)
MAPE: 0.098 (0.073)
R2: -10.147 (18.710)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.395 (0.178)
MAE: 1.085 (0.134)
MAPE: 0.024 (0.003)
R2: 0.701 (0.159)


******
[TEST]
******
RMSE: 7.562 (9.423)
MAE: 4.597 (4.002)
MAPE: 0.109 (0.097)
R2: -16.000 (38.448)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.440 (0.255)
MAE: 1.143 (0.244)
MAPE: 0.026 (0.005)
R2: 0.601 (0.435)


******
[TEST]
******
RMSE: 7.437 (8.112)
MAE: 4.490 (3.610)
MAPE: 0.106 (0.088)
R2: -12.517 (24.844)


Minutes Elapsed:  12.532754604021708


<IPython.core.display.Javascript object>

In [31]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model            Cross Validation Timesteps RMSE Test            \
                                                             mean       std   
0  Chemical  LSTM  Blocking Time Series Split         1  8.855760  2.876757   
1  Chemical  LSTM  Blocking Time Series Split         7  8.606517  4.011269   
2  Chemical  LSTM  Blocking Time Series Split        14  8.183504  3.142711   
3  Chemical  LSTM              Repeated KFold         1  2.124806  0.190481   
4  Chemical  LSTM              Repeated KFold         7  1.991031  0.226499   
5  Chemical  LSTM              Repeated KFold        14  1.869274  0.180537   
6  Chemical  LSTM           Time Series Split         1  6.829251  6.591183   
7  Chemical  LSTM           Time Series Split         7  7.562156  9.423129   
8  Chemical  LSTM           Time Series Split        14  7.436664  8.112493   

   MAE Test           MAPE Test              R2 Test             
       mean       std      mean       std       mean        std  
0  6.512965  1.523500  0.146088  0.032531 -65.473778  74.653616  
1  6.680117  2.959418  0.151559  0.069241 -47.407059  54.516751  
2  7.167373  2.911560  0.160932  0.064122 -44.043854  53.454247  
3  1.540817  0.131379  0.035177  0.003031   0.499794   0.084028  
4  1.560172  0.194801  0.035212  0.004446   0.528595   0.091456  
5  1.465891  0.168536  0.033069  0.003750   0.489726   0.133338  
6  4.221285  3.149332  0.097871  0.072625 -10.147481  18.710451  
7  4.596831  4.002021  0.108660  0.096698 -15.999929  38.447581  
8  4.489936  3.610017  0.106277  0.088465 -12.516991  24.844129

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [32]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()

    def parallel_repeats(repeat):
        set_seeds(SEED + repeat)

        scores_final = None

        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1).copy()
        y = df_copy["CS28"].copy()

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", LSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

        return scores_final

    scores = Parallel(n_jobs=5)(
        delayed(parallel_repeats)(repeat) for repeat in range(repeats)
    )
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += value

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.089 (0.079)
MAE: 1.495 (0.059)
MAPE: 0.034 (0.001)
R2: 0.587 (0.031)


******
[TEST]
******
RMSE: 2.489 (0.308)
MAE: 1.800 (0.116)
MAPE: 0.041 (0.003)
R2: -0.808 (0.458)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.881 (0.466)
MAE: 1.569 (0.460)
MAPE: 0.035 (0.010)
R2: 0.646 (0.177)


******
[TEST]
******
RMSE: 3.345 (0.290)
MAE: 2.731 (0.226)
MAPE: 0.062 (0.005)
R2: -2.303 (0.583)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.260 (0.032)
MAE: 0.974 (0.029)
MAPE: 0.022 (0.001)
R2: 0.850 (0.008)


******
[TEST]
******
RMSE: 2.581 (0.161)
MAE: 2.068 (0.151)
MAPE: 0.047 (0.003)
R2: -1.143 (0.272)


Minutes Elapsed:  4.6220304052035015


<IPython.core.display.Javascript object>

In [33]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation Timesteps RMSE Test            \
                                                              mean       std   
0   Chemical  LSTM  Blocking Time Series Split         1  8.855760  2.876757   
1   Chemical  LSTM  Blocking Time Series Split         7  8.606517  4.011269   
2   Chemical  LSTM  Blocking Time Series Split        14  8.183504  3.142711   
3   Chemical  LSTM           Out of time Split         1  2.488635  0.307616   
4   Chemical  LSTM           Out of time Split         7  3.345059  0.289954   
5   Chemical  LSTM           Out of time Split        14  2.580639  0.160832   
6   Chemical  LSTM              Repeated KFold         1  2.124806  0.190481   
7   Chemical  LSTM              Repeated KFold         7  1.991031  0.226499   
8   Chemical  LSTM              Repeated KFold        14  1.869274  0.180537   
9   Chemical  LSTM           Time Series Split         1  6.829251  6.591183   
10  Chemical  LSTM           Time Series Split         7  7.562156  9.423129   
11  Chemical  LSTM           Time Series Split        14  7.436664  8.112493   

    MAE Test           MAPE Test              R2 Test             
        mean       std      mean       std       mean        std  
0   6.512965  1.523500  0.146088  0.032531 -65.473778  74.653616  
1   6.680117  2.959418  0.151559  0.069241 -47.407059  54.516751  
2   7.167373  2.911560  0.160932  0.064122 -44.043854  53.454247  
3   1.800131  0.115645  0.041009  0.002589  -0.808367   0.458139  
4   2.731170  0.226280  0.061579  0.004937  -2.302653   0.583425  
5   2.067999  0.151012  0.046767  0.003229  -1.143454   0.271995  
6   1.540817  0.131379  0.035177  0.003031   0.499794   0.084028  
7   1.560172  0.194801  0.035212  0.004446   0.528595   0.091456  
8   1.465891  0.168536  0.033069  0.003750   0.489726   0.133338  
9   4.221285  3.149332  0.097871  0.072625 -10.147481  18.710451  
10  4.596831  4.002021  0.108660  0.096698 -15.999929  38.447581  
11  4.489936  3.610017  0.106277  0.088465 -12.516991  24.844129

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [34]:
path = "../../../../../../reports/results/local_models/209/e/full/"
filename = f"lstm_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [35]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/209/e/grouped/"
filename = f"lstm_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>